In [ ]:
import pandas as pd
import tensorflow as tf
import albumentations as A

In [ ]:
df = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')

In [ ]:
t = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')

In [ ]:
t[t.columns[0]]+='.png'

In [ ]:
t.head()

In [ ]:
transform = A.Compose([
    A.Blur(p=0.5,),
    A.Flip(p=0.5),
    A.RandomBrightnessContrast(p=0.1,brightness_limit=1,contrast_limit=1,brightness_by_max=False),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=(0,0.350), rotate_limit=45, p=0.6),
    A.ElasticTransform(p= 0.3,),
    A.GridDistortion(p = 0.3,distort_limit=0.25,interpolation=cv2.INTER_AREA),
    A.HueSaturationValue(p = 0.3,hue_shift_limit=5,sat_shift_limit=6,val_shift_limit=5),
    A.CLAHE(p=0.7,),
    A.CoarseDropout(p = 0.4)
])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
gen = ImageDataGenerator(rotation_range=360,
                               brightness_range=(0,20.0),
                               shear_range=0.2,
                               horizontal_flip=True,
                               vertical_flip=True,)

In [ ]:
path = '../input/aptos2019-blindness-detection/test_images'

test_gen= gen.flow_from_dataframe(t,directory = path,
                                  x_col = t.columns[0],
                                  y_col=None,
                                  batch_size=8,
                                  seed=42,
                                  shuffle=False,
                                  class_mode=None,
                                  target_size=(512,512))

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D, GlobalAveragePooling2D

model=Sequential()
model.add(tf.keras.applications.ResNet152V2(include_top=False,weights=None))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(5,activation='softmax'))

In [ ]:
model1 = tf.keras.models.load_model('../input/new-model/b5_retrained.h5')

In [ ]:
pred = model1.predict(test_gen)

In [ ]:
pred

In [ ]:
def fn(i):
    if i<0.7:
        return 0
    elif 0.7<=i<1.5:
        return 1
    elif 1.5<=i<2.5:
        return 2
    elif 2.5<=i<3.5:
        return 3
    return 4

In [ ]:
def get_ind(l):
    ans = 0
    for i ,v in enumerate(l):
        if l[i]>l[ans]:
            ans = i
    return ans   

In [ ]:
import numpy as np

In [ ]:
ans = [np.argmax(p) for p in pred]

In [ ]:
#ans = [fn(i) for i in pred]
ans

In [ ]:
df.diagnosis = ans

In [ ]:
df.to_csv("./submission.csv", index=False)